<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# FAST-OAD-GA MDA

FAST-OAD-GA is an add-on package for [FAST-OAD framework](https://github.com/fast-aircraft-design/FAST-OAD) for performing rapid Overall Aircraft Design in the category General Aviation (GA). The computational core of FAST-OAD being based on the  [OpenMDAO framework](https://openmdao.org/).

## 1. Setting up and analyzing the initial problem

To organize our work, we propose to use two user folders `data/` and `workdir/`. For instance, in `data/` we store a XML file which describes the Beechcraft Duchess. In `workdir/`, we store files generated or modified by FAST-OAD.

In [1]:
import os.path as pth
import os
import openmdao.api as om
from fastoad import api as api_cs25
from fastga.command import api as api_cs23
import logging
from fastoad.gui import VariableViewer
import shutil

# Define relative path
DATA_FOLDER_PATH = 'data'
WORK_FOLDER_PATH = 'workdir'
OUTPUT_FOLDER_PATH = 'output'

# Clear work folder
shutil.rmtree(WORK_FOLDER_PATH, ignore_errors=True)
os.mkdir(WORK_FOLDER_PATH)

# Define files
CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, 'MDA.yml')
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, 'TBM-900.xml')

# For having log messages on screen
logging.basicConfig(level=logging.INFO, format='%(levelname)-8s: %(message)s')

# For using all screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

Failed to import module fastga.models.weight.mass_breakdown.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.handling_qualities.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.load_analysis.fuselage.compute_bending_moment_reversed_old.py
Failed to import module fastga.models.aerodynamics.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.handling_qualities.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.weight.cg.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.performances.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.geometry.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.weight.cg.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.load_analysis.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.aerodynamics.unitary_tests.test_cirrus_sr22.py
Failed to import module 

After defining a configuration file name, we can ask FAST-GA to generate a default configuration file based on the default OAD model implemented in the framework:

In [2]:
api_cs23.generate_configuration_file(CONFIGURATION_FILE, overwrite=True)

INFO    : Sample configuration written in workdir\MDA.yml


You can now checkout the generated [configuration file](./workdir/MDA.toml). In this configuration file, we have specified an input file name 'problem_inputs.xml'. We can ask FAST-OAD to generate the inputs of the default model with the CeRAS parameters as default values:

In [3]:
api_cs25.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

INFO    : Problem inputs written in C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\notebooks\Lucas\workdir\problem_inputs.xml


'C:\\Users\\Lucas\\FAST-OAD-GA\\FAST-GA\\src\\fastga\\notebooks\\Lucas\\workdir\\problem_inputs.xml'

You can now checkout the generated [input file](./workdir/problem_inputs.xml). As shown previously in the user file architecture, the values in this file can be modified by the user and will be considered by FAST-OAD when executing a computational process.

A useful feature that FAST-OAD provides is to list the outputs of the model defined in the configuration file:

In [ ]:
api_cs25.list_variables(CONFIGURATION_FILE)

Another useful feature is to list the modules of the model defined in the configuration file:

In [ ]:
api_cs25.list_systems(CONFIGURATION_FILE)



.Another useful feature is the [N2 diagram](http://openmdao.org/twodocs/versions/latest/basic_guide/make_n2.html) visualization available in OpenMDAO to see the structure of the model:

In [ ]:
N2_FILE = pth.join(WORK_FOLDER_PATH, 'n2.html')
api_cs25.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=N2_FILE, width='100%', height='500px')

## 2. Running the MDA

Here we run an MDA, that is solving the multidisciplinary couplings using the different nested solvers in the model, without running the optimization problem even if it is defined in the configuration file.

In [4]:
eval_problem = api_cs25.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\aerodynamics\components\compute_propeller_aero.py:406: UserWarning: Propeller element out of calculated polars: contribution canceled!
  warnings.warn("Propeller element out of calculated polars: contribution canceled!")
C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\models\aerodynamics\components\compute_propeller_aero.py:418: UserWarning: Propeller not working in propulsive mode!
  warnings.warn("Propeller not working in propulsive mode!")



aircraft_sizing


NL: NLBGS 1 ; 161074898 1
NL: NLBGS 2 ; 7366387.28 0.045732683
NL: NLBGS 3 ; 6375184.98 0.0395790098
NL: NLBGS 4 ; 1436389.4 0.00891752481
NL: NLBGS 5 ; 883946.83 0.00548780001
NL: NLBGS 6 ; 418061.569 0.00259544828
NL: NLBGS 7 ; 275321.678 0.00170927737
NL: NLBGS 8 ; 139491.142 0.000866001741
NL: NLBGS 9 ; 84768.6983 0.000526268829
NL: NLBGS 10 ; 46009.1471 0.000285638219
NL: NLBGS 11 ; 26397.2557 0.000163881871
NL: NLBGS 12 ; 15057.1111 9.34789424e-05
NL: NLBGS Converged


INFO    : Computation finished after 553.73 seconds
INFO    : Problem outputs written in C:\Users\Lucas\FAST-OAD-GA\FAST-GA\src\fastga\notebooks\Lucas\workdir\problem_outputs.xml


## 3. Running complementary methods

In [5]:
from fastga.models.performances.payload_range import ComputePayloadRange
from fastga.models.load_analysis.loads import Loads
from fastga.models.aerodynamics.external.openvsp.compute_vn import ComputeVNopenvsp


OUTPUT_MDA_FILE = pth.join(WORK_FOLDER_PATH,'problem_outputs.xml')
engine_id = "fastga.wrapper.propulsion.basic_turboprop"
# engine_id = "fastga.wrapper.propulsion.basicIC_engine"


# VN
compute_vn = api_cs23.generate_block_analysis(
        ComputeVNopenvsp(),
        [],
        str(OUTPUT_MDA_FILE),
        True,
    )
output = compute_vn({})

# LOAD ANALYSIS
compute_loads = api_cs23.generate_block_analysis(
        Loads(),
        [],
        str(OUTPUT_MDA_FILE),
        True,
    )
output = compute_loads({})

# Payload Range
compute_payload_range = api_cs23.generate_block_analysis(
        ComputePayloadRange(
            propulsion_id=engine_id,
        ),
        [],
        str(OUTPUT_MDA_FILE),
        True,
)
output = compute_payload_range({})

c:\users\lucas\fast-oad-ga\venvtest\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


Let's save these results. We will use them in the next workbook, that shows some post-processing utilities.

In [6]:
# OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
FINAL_OUTPUT_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_mda_tbm900.xml')
shutil.copy(OUTPUT_MDA_FILE, FINAL_OUTPUT_FILE)

'output\\output_mda_tbm900.xml'

The `variable-viewer` provides a way to inspect the content of the XML file. The dropdown lists above the table allow to filter the displayed variable.

In [ ]:
api_cs25.variable_viewer(FINAL_OUTPUT_FILE)